# Demand analyses
Analyze the demand, both with and without demand response units in terms of
* Overall annual demand as well as
* Demand patterns

## Package imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results,
    plot_single_dispatch_pattern,
)

## Define data sets and global parameters

In [ ]:
# Model configuration in terms of prices and costs
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "50"
dr_scenarios = ["none", "5", "50", "95"]
fuel_price_scenario = "NZE_high"
emissions_pathway = "long-term"
impose_investment_maxima = False

# time frame to visualize
start_time_step = "2020-01-01 00:00:00"
duration_in_time_steps = 168

# inputs and outputs
path_folder_inputs = "./model_inputs/pommesinvest/"
path_folder_results = "./model_results/pommesinvest/"

file_names_inputs = {
    "demand_incl_dr_ts": f"sinks_demand_el_ts_hourly.csv",
    "demand_incl_dr_max": f"sinks_demand_el.csv",
    "demand_excl_dr_ts": f"sinks_demand_el_excl_demand_response_ts_{dr_scenario}_hourly.csv",
    "demand_excl_dr_max": f"sinks_demand_el_excl_demand_response_{dr_scenario}.csv",
    "dr_baseline": f"sinks_demand_response_el_ts_{dr_scenario}.csv",
    "dr_ava_pos": f"sinks_demand_response_el_ava_pos_ts_{dr_scenario}.csv",
    "dr_ava_neg": f"sinks_demand_response_el_ava_neg_ts_{dr_scenario}.csv",
}

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
file_extension = ".csv"

FUELS = ["biomass", "hydrogen", "natgas", "oil", "waste"]
DEMAND_RESPONSE_CLUSTERS = {
    "hoho_cluster_shift_only": "#111111", 
    "hoho_cluster_shift_shed": "#333333", 
    "ind_cluster_shed_only": "#555555",
    "ind_cluster_shift_only": "#999999", 
    "ind_cluster_shift_shed": "#aaaaaa", 
    "tcs+hoho_cluster_shift_only": "#cccccc",
    "tcs_cluster_shift_only": "#dddddd", 
}
NAMES = {
    "baseline": "baseline demand", 
    "lower_limit": "lower demand limit",
    "upper_limit": "upper demand limit",    
}
STYLE = {
    "colors": {
        NAMES["baseline"]: "black",
        NAMES["lower_limit"]: "blue",
        NAMES["upper_limit"]: "red"
    },
    "linestyles": {
        NAMES["baseline"]: "-",
        NAMES["lower_limit"]: "--",
        NAMES["upper_limit"]: "-."
    }
}

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file_names = {
    "demand_incl_dr_ts": "demand_incl_dr_annual",
    "demand_excl_dr_ts": "demand_exl_dr_annual",
}

plt.rcParams.update({'font.size': 14})

# Annual demand inspection
## Read in and preprocess demand data

In [ ]:
data_sets = {
    data_set: pd.read_csv(f"{path_folder_inputs}{file_name}", index_col=0) 
    for data_set, file_name in file_names_inputs.items()
}

In [ ]:
# Slice 2020 data in the first place
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[f"{key}_DE_2020"] = data_sets[key].loc[
        data_sets[key].index.str[:4] == "2020", 
        [col for col in data_sets[key].columns if "DE_" in col]
    ]

## Combine to annual values

In [ ]:
map_ts_to_max = {
    "demand_incl_dr_ts": data_sets["demand_incl_dr_max"],
    "demand_excl_dr_ts": data_sets["demand_excl_dr_max"],
}

In [ ]:
# Create annual data sets
for key in ["demand_incl_dr_ts", "demand_excl_dr_ts"]:
    data_sets[key].index = data_sets[key].index.str[:4]
    data_sets[f"{key}_annual"] = data_sets[key].groupby(data_sets[key].index).sum()["DE_sink_el_load"]
    data_sets[f"{key}_annual"] = data_sets[f"{key}_annual"].mul(
        map_ts_to_max[key].at["DE_sink_el_load", "maximum"]
    ).div(1000)
    data_sets[f"{key}_annual"].to_csv(f"{path_data_out}{output_file_names[key]}.csv")

In [ ]:
# Plot annual demand without demand response
fig, ax = plt.subplots(figsize=(12, 5))
_ = data_sets["demand_incl_dr_ts_annual"].loc["2020":"2045"].plot(kind="bar", ax=ax, color="darkblue")
_ = plt.xlabel("year")
_ = plt.ylabel("annual demand in GWh")
#_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_incl_dr_ts']}.png", dpi=300)
plt.show()
plt.close()

In [ ]:
data_sets["demand_excl_dr_ts_annual"]

In [ ]:
# Read in demand response potential parameter data and extract maximum value
dr_potential_data = {
    dr_cluster: pd.read_csv(
        f"{path_folder_inputs}{dr_cluster}_potential_parameters_{dr_scenario}%.csv", 
        index_col=0
    )
    for dr_cluster in DEMAND_RESPONSE_CLUSTERS
}
dr_max_potentials = {}
for key, val in dr_potential_data.items():
    dr_max_potentials[key] = val["max_cap"]

# Calculate annual demand (assumed constant here)
data_sets["dr_baseline_yearly"] = data_sets["dr_baseline"].copy()
data_sets["dr_baseline_yearly"].index = data_sets["dr_baseline_yearly"].index.str[:4]

data_sets["dr_annual_demand"] = pd.DataFrame()
for dr_cluster in DEMAND_RESPONSE_CLUSTERS:
    data_sets["dr_annual_demand"][dr_cluster] = dr_max_potentials[dr_cluster].mul(
        data_sets["dr_baseline_yearly"].groupby(
            data_sets["dr_baseline_yearly"].index
        ).sum().at["2020", dr_cluster]
    ).div(1000)

data_sets["dr_annual_demand"].index = data_sets["dr_annual_demand"].index.astype(str)
    
# Combine with demand data set excluding demand response
data_sets["demand_excl_dr_ts_annual"] = pd.concat(
    [data_sets["demand_excl_dr_ts_annual"], data_sets["dr_annual_demand"]],
    axis=1
)
data_sets["demand_excl_dr_ts_annual"].rename(
    columns={"DE_sink_el_load": "load without demand response"},
    inplace=True
)
colors = {
    "load without demand response": "darkblue",
    **DEMAND_RESPONSE_CLUSTERS
}

In [ ]:
# Plot annual demand excl demand response with demand response baseline demand on top
fig, ax = plt.subplots(figsize=(12, 5))
data_sets["demand_excl_dr_ts_annual"] = data_sets["demand_excl_dr_ts_annual"][[col for col in colors]]
_ = data_sets["demand_excl_dr_ts_annual"].loc["2020":"2045"].plot(kind="bar", edgecolor="darkgrey", stacked=True, ax=ax, color=colors)
_ = plt.xlabel("year")
_ = plt.ylabel("annual demand in GWh")
_ = plt.legend(bbox_to_anchor=[1.02, 1.02])
current_values = plt.gca().get_yticks()
_ = plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])
plt.tight_layout()
plt.savefig(f"{path_plots}{output_file_names['demand_excl_dr_ts']}.png", dpi=300)
plt.show()
plt.close()

# Demand patterns inspection
## Extract demand response investments results

In [ ]:
investment_results_raw = pd.read_csv(f"{path_folder_results}{filename}{file_add_on}{file_extension}", index_col=0)
processed_results = preprocess_raw_results(investment_results_raw)
aggregated_results, other_storage_results = aggregate_investment_results(
    processed_results, energy_carriers=FUELS, by="energy_carrier"
)

demand_response_investments = aggregated_results.loc[
    aggregated_results.index.get_level_values(0).isin(DEMAND_RESPONSE_CLUSTERS), "total"
]

## Calculate profiles

Extract the absolute values of
* baseline profile,
* economic downshift potential and
* economic upshift potential.

Calculate the absolute demand bounds from downshift resp. upshift potentials.

> _Note:_
> * _Downshift is limited to baseline profile._
> * _This is implicitly done in pommesinvest by setting lower bound of flows to 0_

In [ ]:
profiles = {}
for dr_cluster in DEMAND_RESPONSE_CLUSTERS:
    profiles[dr_cluster] = pd.DataFrame(
        columns=["baseline", "downshift_potential", "upshift_potential"]
    )
    to_concat = {k: [] for k in profiles[dr_cluster].columns}
    
    # Calculate absolute values using annual max capacity resp. potentials
    for iter_year in range(2020, 2046):
        to_concat["baseline"].append(
            data_sets["dr_baseline"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(dr_max_potentials[dr_cluster].loc[iter_year])
        )
        to_concat["downshift_potential"].append(
           data_sets["dr_ava_pos"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(demand_response_investments.loc[dr_cluster, str(iter_year)])
        )
        to_concat["upshift_potential"].append(
           data_sets["dr_ava_neg"].loc[
                f"{iter_year}-01-01 00:00": f"{iter_year}-12-31 23:59", dr_cluster
            ].mul(demand_response_investments.loc[dr_cluster, str(iter_year)])
        ) 

    for col in profiles[dr_cluster].columns:
        profiles[dr_cluster][col] = pd.concat(to_concat[col])
    
    # Limit downshift potential
    profiles[dr_cluster]["downshift_potential"] = np.where(
        profiles[dr_cluster]["baseline"] < profiles[dr_cluster]["downshift_potential"],
        profiles[dr_cluster]["baseline"],
        profiles[dr_cluster]["downshift_potential"]
    )
    profiles[dr_cluster]["lower_limit"] = profiles[dr_cluster]["baseline"] - profiles[dr_cluster]["downshift_potential"]
    profiles[dr_cluster]["upper_limit"] = profiles[dr_cluster]["baseline"] + profiles[dr_cluster]["upshift_potential"]
    
    profiles[dr_cluster] = profiles[dr_cluster][[col for col in NAMES]].rename(columns=NAMES)

## Plot absolute profiles (baseline and flexibility band)

In [ ]:
for dr_cluster in DEMAND_RESPONSE_CLUSTERS:
    plot_single_dispatch_pattern(
        profiles[dr_cluster],
        start_time_step=start_time_step,
        amount_of_time_steps=duration_in_time_steps,
        colors=STYLE["colors"],
        save=True,
        path_plots="./plots/",
        filename=f"absolute_potential_{dr_cluster}",
        kind="line",
        stacked=None,
        figsize=(15, 10),
        linestyle=STYLE["linestyles"],
        title="Flexibility bands",
        ylabel="Power [MW]",
    )

## Calculate and plot relative profiles
For normalization, define maximum of baseline demand of respective time frame to equal 1

In [ ]:
# get end index to derive time frame for normalization
index_start = int(profiles[dr_cluster].index.get_loc(start_time_step))
index_end = int(index_start + duration_in_time_steps)
end_time_step = profiles[dr_cluster].iloc[index_end].name

relative_profiles = {}
for dr_cluster in DEMAND_RESPONSE_CLUSTERS:
    relative_profiles[dr_cluster] = (
        profiles[dr_cluster].iloc[index_start : index_end].div(
            profiles[dr_cluster].iloc[index_start : index_end]["baseline demand"].max()
        )
    )
    
    plot_single_dispatch_pattern(
        relative_profiles[dr_cluster],
        start_time_step=start_time_step,
        amount_of_time_steps=duration_in_time_steps - 2,
        colors=STYLE["colors"],
        save=True,
        path_plots="./plots/",
        filename=f"relative_potential_{dr_cluster}",
        kind="line",
        stacked=None,
        figsize=(15, 10),
        linestyle=STYLE["linestyles"],
        title="Flexibility bands",
        ylabel="Power [MW]",
    )